In [1]:
from collections import defaultdict
import os
import sys

import h5py
import numpy as np
import scanpy as sc
import scipy.sparse
import torch
import wandb

sys.path.insert(0, os.path.abspath('../examples'))
import data
import celltrip

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
BASE_FOLDER = os.path.abspath('')
DATA_FOLDER = os.path.join(BASE_FOLDER, '../data')

- TODO
  - Tag ouputs by `run_id`

## Data Setup

In [2]:
# Get run
run_id = '32jqyk54'
api = wandb.Api()
run = api.run(f'oafish/cellTRIP/{run_id}')
config = defaultdict(lambda: {})
for k, v in run.config.items():
    dict_name, key = k.split('/')
    config[dict_name][key] = v
config = dict(config)

# Parameters
dataset_name = config['data']['dataset']
imputation_target = config['env']['reward_distance_target']
dimensions = config['env']['dim']
notebook_seed = np.random.randint(2**32)

# Apply seed
torch.manual_seed(notebook_seed)
if torch.cuda.is_available(): torch.cuda.manual_seed(notebook_seed)
np.random.seed(notebook_seed)

# Derivatives
RUN_FOLDER = os.path.join(BASE_FOLDER, 'runs', dataset_name)

# Load data and save to file
modalities, types, features = data.load_data(dataset_name, DATA_FOLDER)
ppc = celltrip.utilities.Preprocessing(**config['data']).fit(modalities)  # Potentially mismatched if sampled
modalities, types = ppc.transform(modalities, types)

if not os.path.exists(RUN_FOLDER): os.makedirs(RUN_FOLDER)
for i in range(len(modalities)):
    modality = modalities[i]

    # Regular matrices (ManiNetCluster, JAMIE)
    np.savetxt(os.path.join(RUN_FOLDER, f'X{i+1}.txt'), modality, delimiter='\t')

    # Similarity matrices (MMD-MA)
    modality_z = modality - modality.mean(axis=0, keepdims=True) / modality.std(axis=0, keepdims=True)
    similarity = np.matmul(modality_z, modality_z.T)
    np.savetxt(os.path.join(RUN_FOLDER, f'X{i+1}_sim.tsv'), similarity, delimiter='\t')

    # Anndata matrices (scVI)
    # adata = sc.AnnData(modalities[0])
    # adata.var_names = features[0] if isinstance(features[0][0], str) else [f'Feature_{fi}' for fi in features[0]]
    # adata.obs_names = [f'Cell_{j}' for j in range(len(adata.obs_names))]
    # adata.obs['cell_type'] = types[0][:, 0]
    # adata.obs['time'] = types[0][:, -1]
    # # adata.obs['batch'] = 0
    # adata.write(os.path.join(RUN_FOLDER, f'X{i+1}.h5ad'), compression='gzip')

    # HDFS
    # https://github.com/scverse/anndata/issues/595#issuecomment-1824376236
    concatenated_modalities = np.concatenate(modalities, axis=-1)
    barcodes = [f'Cell {i}' for i in range(concatenated_modalities.shape[0])]
    feature_types = modalities[0].shape[1] * ['Gene Expression'] + modalities[1].shape[1] * ['Peaks']
    feature_names = np.concatenate(features)
    feature_ids = np.array(np.arange(feature_names.shape[0]), dtype='str')
    genome = concatenated_modalities.shape[1] * ['Something']
    sparse_data = scipy.sparse.csr_matrix(concatenated_modalities)

    def int_max(x):
        return int(max(np.floor(len(str(int(max(x)))) / 4), 1) * 4)
    def str_max(x):
        return max([len(i) for i in x])

    with h5py.File(os.path.join(RUN_FOLDER, f'X.h5'), 'w') as f:
        grp = f.create_group('matrix')
        grp.create_dataset('barcodes', data=np.array(barcodes, dtype=f'|S{str_max(barcodes)}'))
        grp.create_dataset('data', data=np.array(sparse_data.data, dtype=f'<i{int_max(sparse_data.data)}'))
        ftrs = grp.create_group('features')
        # # this group will lack the following keys:
        # # '_all_tag_keys', 'feature_type', 'genome', 'id', 'name', 'pattern', 'read', 'sequence'
        ftrs.create_dataset('feature_type', data=np.array(feature_types, dtype=f'|S{str_max(feature_types)}'))
        ftrs.create_dataset('genome', data=np.array(genome, dtype=f'|S{str_max(genome)}'))
        ftrs.create_dataset('id', data=np.array(feature_ids, dtype=f'|S{str_max(feature_ids)}'))
        ftrs.create_dataset('name', data=np.array(feature_names, dtype=f'|S{str_max([str(fn) for fn in feature_names])}'))
        grp.create_dataset('indices', data=np.array(sparse_data.indices, dtype=f'<i{int_max(sparse_data.indices)}'))
        grp.create_dataset('indptr', data=np.array(sparse_data.indptr, dtype=f'<i{int_max(sparse_data.indptr)}'))
        grp.create_dataset('shape', data=np.array(sparse_data.shape[::-1], dtype=f'<i{int_max(sparse_data.shape)}'))

# Preview h5 files
# print('Generated File')
# with h5py.File(os.path.join(RUN_FOLDER, f'X.h5'), 'r') as f: celltrip.utilities.h5_tree(f)
# print('\nBaseline File')
# with h5py.File('/home/thema/Downloads/DM_rep4.h5', 'r') as f: celltrip.utilities.h5_tree(f)

## Integration Methods

In [3]:
if imputation_target is None:
    # LMA
    # https://github.com/namtk/ManiNetCluster/tree/master/inst/python
    method_name = 'LMA'
    print(f'Running {method_name}')
    new_wd = os.path.join(RUN_FOLDER, method_name)
    if not os.path.exists(new_wd): os.makedirs(new_wd)
    os.chdir(new_wd)

    !conda run -n maninetcluster \
    python {os.path.join(BASE_FOLDER, 'maninetcluster_helper.py')} \
    {os.path.join(RUN_FOLDER, 'X1.txt')} \
    {os.path.join(RUN_FOLDER, 'X2.txt')} \
    --align lma \
    -p {dimensions}

    os.chdir(BASE_FOLDER)


    # CCA
    # https://github.com/namtk/ManiNetCluster/tree/master/inst/python
    method_name = 'CCA'
    print(f'Running {method_name}')
    new_wd = os.path.join(RUN_FOLDER, method_name)
    if not os.path.exists(new_wd): os.makedirs(new_wd)
    os.chdir(new_wd)

    !conda run -n maninetcluster \
    python {os.path.join(BASE_FOLDER, 'maninetcluster_helper.py')} \
    {os.path.join(RUN_FOLDER, 'X1.txt')} \
    {os.path.join(RUN_FOLDER, 'X2.txt')} \
    --align cca \
    -p {dimensions}

    os.chdir(BASE_FOLDER)


    # NLMA
    # https://github.com/namtk/ManiNetCluster/tree/master/inst/python
    method_name = 'NLMA'
    print(f'Running {method_name}')
    new_wd = os.path.join(RUN_FOLDER, method_name)
    if not os.path.exists(new_wd): os.makedirs(new_wd)
    os.chdir(new_wd)

    !conda run -n maninetcluster \
    python {os.path.join(BASE_FOLDER, 'maninetcluster_helper.py')} \
    {os.path.join(RUN_FOLDER, 'X1.txt')} \
    {os.path.join(RUN_FOLDER, 'X2.txt')} \
    --align nlma \
    -p {dimensions}

    os.chdir(BASE_FOLDER)


    # JAMIE
    method_name = 'JAMIE'
    print(f'Running {method_name}')
    new_wd = os.path.join(RUN_FOLDER, method_name)
    if not os.path.exists(new_wd): os.makedirs(new_wd)
    os.chdir(new_wd)

    !conda run -n jamie \
    python {os.path.join(BASE_FOLDER, 'jamie_helper.py')} \
    {os.path.join(RUN_FOLDER, 'X1.txt')} \
    {os.path.join(RUN_FOLDER, 'X2.txt')} \
    -p {dimensions} \
    -s {notebook_seed} \
    --suffix {notebook_seed}

    os.chdir(BASE_FOLDER)


    # MMD-MA
    # https://bitbucket.org/noblelab/2019_mmd_wabi/src/master/manifoldAlignDistortionPen_mmd_multipleStarts.py
    method_name = 'MMD-MA'
    print(f'Running {method_name}')
    new_wd = os.path.join(RUN_FOLDER, method_name)
    if not os.path.exists(new_wd): os.makedirs(new_wd)
    os.chdir(new_wd)

    fname1, fname2 = f'alpha_hat_{notebook_seed}_10000.txt', f'beta_hat_{notebook_seed}_10000.txt'
    !conda run -n mmdma \
    python {os.path.join(BASE_FOLDER, '2019_mmd_wabi/manifoldAlignDistortionPen_mmd_multipleStarts.py')} \
    {os.path.join(RUN_FOLDER, 'X1_sim.tsv')} \
    {os.path.join(RUN_FOLDER, 'X2_sim.tsv')} \
    --seed {notebook_seed} \
    --p {dimensions}
    !python {os.path.join(BASE_FOLDER, 'mmd_helper.py')} \
    {fname1} \
    {fname2} \
    --suffix {notebook_seed}

    os.chdir(BASE_FOLDER)

## Imputation Methods

In [4]:
if imputation_target is not None:
    # KNN
    method_name = 'KNN'
    print(f'Running {method_name}')
    new_wd = os.path.join(RUN_FOLDER, method_name)
    if not os.path.exists(new_wd): os.makedirs(new_wd)
    os.chdir(new_wd)

    import sklearn.neighbors

    # Load data
    X1, X2 = np.loadtxt('../X1.txt'), np.loadtxt('../X2.txt')
    dataset = [X1, X2]
    X, Y = dataset[-imputation_target], dataset[imputation_target]

    # Fit model
    knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=10)
    knn.fit(X, Y)
    projection = [None for _ in range(2)]
    projection[imputation_target] = knn.predict(X)

    # Write to file
    for i, proj in enumerate(projection):
        if proj is not None: np.savetxt(f'I{i+1}.txt', proj)

    os.chdir(BASE_FOLDER)


    # MLP
    method_name = 'MLP'
    print(f'Running {method_name}')
    new_wd = os.path.join(RUN_FOLDER, method_name)
    if not os.path.exists(new_wd): os.makedirs(new_wd)
    os.chdir(new_wd)

    import sklearn.neighbors
    import sklearn.neural_network

    # Load data
    X1, X2 = np.loadtxt('../X1.txt'), np.loadtxt('../X2.txt')
    dataset = [X1, X2]
    X, Y = dataset[-imputation_target], dataset[imputation_target]

    # Fit model
    np.random.seed(notebook_seed)
    mlp = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(128,), max_iter=1_000)
    mlp.fit(X, Y)
    projection = [None for _ in range(2)]
    projection[imputation_target] = mlp.predict(X)

    # Write to file
    for i, proj in enumerate(projection):
        if proj is not None: np.savetxt(f'I{i+1}_{notebook_seed}.txt', proj)

    os.chdir(BASE_FOLDER)


    # JAMIE
    # https://github.com/Oafish1/JAMIE
    method_name = 'JAMIE'
    print(f'Running {method_name}')
    new_wd = os.path.join(RUN_FOLDER, method_name)
    if not os.path.exists(new_wd): os.makedirs(new_wd)
    os.chdir(new_wd)

    !conda run -n jamie \
    python {os.path.join(BASE_FOLDER, 'jamie_helper.py')} \
    {os.path.join(RUN_FOLDER, 'X1.txt')} \
    {os.path.join(RUN_FOLDER, 'X2.txt')} \
    -t {imputation_target+1} \
    -p {dimensions} \
    -s {notebook_seed} \
    --suffix {notebook_seed}

    os.chdir(BASE_FOLDER)

        
    # BABEL
    # https://github.com/wukevin/babel
    if False:
        method_name = 'BABEL'
        print(f'Running {method_name}')
        new_wd = os.path.join(RUN_FOLDER, method_name)
        if not os.path.exists(new_wd): os.makedirs(new_wd)
        os.chdir(new_wd)

        !conda run -n babel \
        python {os.path.join(BASE_FOLDER, 'babel/bin/train_model.py')} \
        --data {os.path.join(RUN_FOLDER, 'X.h5')} \
        --outdir {new_wd}
        !conda run -n babel \
        python {os.path.join(BASE_FOLDER, 'babel/bin/predict_model.py')} \
        --checkpoint {os.path.join(new_wd, net_asdf)} \
        --data {os.path.join(RUN_FOLDER, 'X.h5')} \
        --outdir {new_wd}

        os.chdir(BASE_FOLDER)


    # scVI
    if False:  # Not done
        method_name = 'scVI'
        print(f'Running {method_name}')
        new_wd = os.path.join(RUN_FOLDER, method_name)
        if not os.path.exists(new_wd): os.makedirs(new_wd)
        os.chdir(new_wd)


        X_fname = os.path.join(RUN_FOLDER, 'X1.txt')
        Y_fname = os.path.join(RUN_FOLDER, 'X2.txt')

        import numpy as np
        import scvi

        scvi.settings.seed = 42

        X = np.loadtxt(X_fname)
        Y = np.loadtxt(Y_fname)

        scvi.model.SCVI.setup_anndata


        os.chdir(BASE_FOLDER)

Running KNN
Running MLP


Running JAMIE


use random seed: 1504220495
Shape of Raw data
Dataset 0: (2150, 253)
Dataset 1: (2150, 2)
Device: cuda:0
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:0.0181 alpha:0.0009
epoch:[1000/2000] err:0.0000 alpha:0.0000
epoch:[1500/2000] err:0.0771 alpha:0.0040
epoch:[2000/2000] err:0.2887 alpha:0.0149
Finished Matching!
---------------------------------
Train coupled autoencoders
epoch:[500/10000]: loss:2.892051
epoch:[1000/10000]: loss:2.216013
epoch:[1500/10000]: loss:2.038872
epoch:[2000/10000]: loss:1.974083
epoch:[2500/10000]: loss:1.841926
epoch:[3000/10000]: loss:1.902894
Finished Mapping!
---------------------------------
JAMIE Done!
Distance: 7.683406418189406
Correspondence: 80.78843666799366
Mapping: 145.05275040492415
Total: 233.52459349110723


/home/thema/miniconda3/envs/jamie/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit

## Perturbation Methods

In [5]:
# Variance
# https://shap.readthedocs.io/en/latest/
method_name = 'variance'
print(f'Running {method_name}')
new_wd = os.path.join(RUN_FOLDER, method_name)
if not os.path.exists(new_wd): os.makedirs(new_wd)
os.chdir(new_wd)

 # Load data
X1, X2 = np.loadtxt(os.path.join(RUN_FOLDER, 'X1.txt')), np.loadtxt(os.path.join(RUN_FOLDER, 'X2.txt'))
dataset = [X1, X2]

# Get variance
importance = [np.var(X, axis=0) for X in dataset]
importance = [imp / imp.sum() for imp in dataset]

# Write to file
for i, imp in enumerate(importance):
    if imp is not None: np.savetxt(f'F{i+1}_{notebook_seed}.txt', imp)

os.chdir(BASE_FOLDER)

Running variance


## Trajectory Methods